In [84]:
"""
this file is demo file for prototyping python functions
"""

'\nthis file is demo file for prototyping python functions\n'

In [41]:
import json
import requests
import random
api_key = '3a3bd8764d180fafd01e23bafc751a09'
base_url= 'https://v3.football.api-sports.io/'



In [39]:
def fetch_fixtures():


    endpoint = "/fixtures"
    url =f"{base_url+endpoint}"
    querystring = {"date":"2025-06-08"}

    headers = {'x-rapidapi-host': 'v3.football.api-sports.io',
                 "x-rapidapi-key":api_key
               }

    response = requests.get(url, headers=headers, params=querystring)
    print(response.status_code)
    print(json.dumps(response.json(), ensure_ascii=False, indent=2))
    
    return   response.json()["response"]


In [40]:
fixtures=fetch_fixtures()

200
{
  "get": "fixtures",
  "parameters": {
    "date": "2025-06-08"
  },
  "errors": [],
  "results": 464,
  "paging": {
    "current": 1,
    "total": 1
  },
  "response": [
    {
      "fixture": {
        "id": 1247818,
        "referee": null,
        "timezone": "UTC",
        "date": "2025-06-08T00:00:00+00:00",
        "timestamp": 1749340800,
        "periods": {
          "first": null,
          "second": null
        },
        "venue": {
          "id": 5860,
          "name": "Sportplatz SV Dellach/Gail",
          "city": "Dellach/Gail"
        },
        "status": {
          "long": "Match Cancelled",
          "short": "CANC",
          "elapsed": null,
          "extra": null
        }
      },
      "league": {
        "id": 225,
        "name": "Landesliga - Karnten",
        "country": "Austria",
        "logo": "https://media.api-sports.io/football/leagues/225.png",
        "flag": "https://media.api-sports.io/flags/at.svg",
        "season": 2024,
        "roun

In [42]:
limited_fixtures = random.sample(fixtures, 100)  # pick 100 unique random fixtures
#len(fixtures)

In [43]:
print(limited_fixtures)

[{'fixture': {'id': 1326822, 'referee': None, 'timezone': 'UTC', 'date': '2025-06-08T03:00:00+00:00', 'timestamp': 1749351600, 'periods': {'first': 1749351600, 'second': 1749355200}, 'venue': {'id': None, 'name': 'Lancaster Municipal Stadium', 'city': 'Lancaster, California'}, 'status': {'long': 'Match Finished', 'short': 'FT', 'elapsed': 90, 'extra': 6}}, 'league': {'id': 489, 'name': 'USL League One', 'country': 'USA', 'logo': 'https://media.api-sports.io/football/leagues/489.png', 'flag': 'https://media.api-sports.io/flags/us.svg', 'season': 2025, 'round': 'Regular Season - 7', 'standings': True}, 'teams': {'home': {'id': 25838, 'name': 'Alta', 'logo': 'https://media.api-sports.io/football/teams/25838.png', 'winner': None}, 'away': {'id': 22785, 'name': 'Spokane Velocity', 'logo': 'https://media.api-sports.io/football/teams/22785.png', 'winner': None}}, 'goals': {'home': 0, 'away': 0}, 'score': {'halftime': {'home': 0, 'away': 0}, 'fulltime': {'home': 0, 'away': 0}, 'extratime': {'h

In [64]:
def  extract_teams_id(fixtures):
    teams_ids=set()
    metadata=[]
    
    for fixture in fixtures:
        home_team = fixture["teams"]["home"]["id"]
        away_team = fixture["teams"]["away"]["id"]
        league_id = fixture["league"]["id"]
        season = fixture["league"]["season"]

        teams_ids.update([home_team, away_team])

        metadata.append({
            "home_team": home_team,
            "away_team": away_team,
            "league_id": league_id,
            "season": season
        })
        # print(list(teams_ids))

    return list(teams_ids), metadata 
    



In [65]:
teams_IDs,metadata=extract_teams_id(limited_fixtures)

In [66]:
print(len(teams_IDs))
print(len(limited_fixtures))
print(len(metadata))

200
100
100


In [69]:
print(teams_IDs[:10])

[4099, 24580, 1030, 18450, 12307, 12308, 6676, 17429, 4119, 19476]


#### **build team requests**

In [48]:
def build_team_requests(teams, metadata):
    team_requests = set()

    for match in metadata:
        for team_id in [match["home_team"], match["away_team"]]:
            if team_id in teams:
                team_requests.add((
                    team_id,
                    match["league_id"],
                    match["season"]
                ))

    return list(team_requests)

In [49]:
team_requests=build_team_requests(teams_IDs,metadata)

In [60]:
print(len(team_requests))

200


In [70]:
def fetch_team_info(team_id):
    endpoint = "/teams"
    url =f"{base_url+endpoint}"
    headers = {'x-rapidapi-host': 'v3.football.api-sports.io',
                 "x-rapidapi-key":api_key
               }

    querystring = {"id":team_id}

    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed for team {team_id}")
        return None

In [78]:
import time

def extract_team_info_from_fixtures(team_requests):
    all_team_stats = []

    for team_id in team_requests:
        info = fetch_team_info(team_id)
        if info:
            all_team_stats.append(info)

        time.sleep(6)  # ⏱️ Wait 6 seconds to respect 10 reqs/minute

    return all_team_stats


In [79]:
team_info = extract_team_info_from_fixtures(teams_IDs)

In [83]:
print(len(team_info))
team_info[:10]

200


[{'get': 'teams',
  'parameters': {'id': '4099'},
  'errors': [],
  'results': 1,
  'paging': {'current': 1, 'total': 1},
  'response': [{'team': {'id': 4099,
     'name': 'Westchester Flames',
     'code': None,
     'country': 'USA',
     'founded': 1999,
     'national': False,
     'logo': 'https://media.api-sports.io/football/teams/4099.png'},
    'venue': {'id': 3142,
     'name': 'Skidelsky Field at City Park Stadium',
     'address': '5th Avenue &amp; Potter Street, New Rochelle',
     'city': 'New York',
     'capacity': 1845,
     'surface': 'grass',
     'image': 'https://media.api-sports.io/football/venues/3142.png'}}]},
 {'get': 'teams',
  'parameters': {'id': '24580'},
  'errors': [],
  'results': 1,
  'paging': {'current': 1, 'total': 1},
  'response': [{'team': {'id': 24580,
     'name': 'Porreres',
     'code': None,
     'country': 'Spain',
     'founded': None,
     'national': False,
     'logo': 'https://media.api-sports.io/football/teams/24580.png'},
    'venue': 